In [1]:
import numpy as np
import pandas as pd
import healpy as hp
import matplotlib
from matplotlib import pyplot as plt

In [2]:
#read in the new dataframe for the filter
df = pd.read_csv('Z087_files_data.csv')

In [3]:
def get_vertices(index):
    ra1 = df['RA1'][index]
    ra2 = df['RA2'][index]
    ra3 = df['RA3'][index]
    ra4 = df['RA4'][index]
    dec1 = df['DEC1'][index]
    dec2 = df['DEC2'][index]
    dec3 = df['DEC3'][index]
    dec4 = df['DEC4'][index]
    def ra_dec_to_theta_phi(ra, dec):
        ra_rad = np.radians(ra)
        dec_rad = np.radians(dec)
        theta = np.pi / 2 - dec_rad
        phi = ra_rad
        return theta, phi
    theta1, phi1 = ra_dec_to_theta_phi(ra1, dec1)
    theta2, phi2 = ra_dec_to_theta_phi(ra2, dec2)
    theta3, phi3 = ra_dec_to_theta_phi(ra3, dec3)
    theta4, phi4 = ra_dec_to_theta_phi(ra4, dec4)
    vec1 = hp.ang2vec(theta1, phi1)
    vec2 = hp.ang2vec(theta2, phi2)
    vec3 = hp.ang2vec(theta3, phi3)
    vec4 = hp.ang2vec(theta4, phi4)
    vertices = np.array([vec1, vec2, vec3, vec4])
    return vertices

#### For getting the day by day exposure maps.

In [ ]:
#Convert the MJD float value to an integer for slicing.
df['MJD']= df['MJD'].astype(int)
days = np.unique(df['MJD'].values)

In [4]:
for i in range(0, len(days)):   #iterate through each day
    day = days[i]
    obs_rows = df[(df['MJD'] == day)].index #get the rows for one footprint
    NSIDE = 4096
    NPIX  = hp.nside2npix(NSIDE)
    ipix_box_list = [] #get the pixel list for plotting the SCAs
    for index in obs_rows:
        vertices = get_vertices(index)
        ipix_box_single = hp.query_polygon(nside=NSIDE, vertices=vertices, inclusive=False) #make the array of pixels for the projected SCAs
        ipix_box_list.append(ipix_box_single)
    ipix_box = np.concatenate(ipix_box_list)
    rot = [9.7, -44] #point to center the projection on
    m = np.zeros(hp.nside2npix(NSIDE)) #map for the plot
    counts = np.bincount(ipix_box, minlength=len(m))
    m[:len(counts)] = counts
    hp.gnomview(m, rot=rot,title="Sky Location of Roman Pictures-Z087 Band", reso = 1.6, xsize = 200) #plot the projection
    plt.savefig(f'/mnt/c/Users/alexj/Downloads/Z087 images/Z087_plot_{i+1}.png') #save the image
    plt.close() #close plots to conserve memory

#### For getting the overall final exposure map

In [ ]:
obs_rows = df.index 
ipix_box_list = []
for index in obs_rows:
    vertices = get_vertices(index)
    ipix_box_single = hp.query_polygon(nside=NSIDE, vertices=vertices, inclusive=False)
    ipix_box_list.append(ipix_box_single)
ipix_box = np.concatenate(ipix_box_list)
rot = [9.7, -44]
m = np.zeros(hp.nside2npix(NSIDE))
counts = np.bincount(ipix_box, minlength=len(m))
m[:len(counts)] = counts
hp.gnomview(m, rot=rot,title="Sky Location of Roman Pictures-Z087 Band", reso = 1.6, xsize = 200)
plt.savefig(f'/mnt/c/Users/alexj/Downloads/Z087 images/Z087_plot_all.png')
plt.close()